In [ ]:
import pandas as pd
import psycopg2

In [ ]:
DB_HOST = '89.145.163.87'
DB_PORT = 5432
DB_USERNAME = 'group11'
DB_PASSWORD = 'tsw2020'
DB_NAME = 'tweets'

In [ ]:
def db_to_pandas(query):
    """ Query database to a Pandas DF"""
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USERNAME, password=DB_PASSWORD)
    cur = conn.cursor()
    
    df = pd.read_sql_query(query, conn)
    print(df.info)
    
    cur.close()
    conn.close()
    
    return df

In [ ]:
sql_query = """
SELECT tweet_id, concepts, tweet_body->>'created_at' as created_at
FROM tweet
WHERE location_id LIKE 'germany%'
AND concepts IS NOT NULL;
"""

df_from_db = db_to_pandas(sql_query)

In [ ]:
# convert to timestamps to days
df_from_db.date = pd.to_datetime(df_from_db['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
df_from_db.date = df_from_db.date.dt.normalize()

df_from_db.info

In [ ]:
a = df_from_db.explode('concepts')
concept_df = (a.assign(concepts = a['concepts'].str.get('surfaceForm'))\
              .groupby(['date','concepts'])['concepts'].count().reset_index(name='count'))
print(concept_df)

In [ ]:
top_concepts = concept_df.groupby(['concepts']).agg({'count':sum}).sort_values(['count'], ascending=False)
top_concepts.info

In [ ]:
for i, c in top_concepts.iterrows():
    print(c)

In [ ]:
stop = ('coronavirus', 'Coronavirus', 'Corona', 'CoronaVirus', 'do', 'amp', 'Deutschland', 'via', 'tun', 'primer', 'un', 'del')
relevant_top = concept_df[~concept_df.concepts.isin(stop)]
relevant_top.groupby(['concepts']).agg({'count':sum}).sort_values(['count'], ascending=False)

In [ ]:
interesting = ['Italien', 'China', 'Jens Spahn', 
               'Desinfektionsmittel', 'Mundschutz',
               'Robert-Koch-Institut', 'Iran', 'Italy'
              ]

interesting_df = concept_df[concept_df.concepts.isin(interesting)]
interesting_df.info

In [ ]:
sql_query = """
SELECT tweet_body->>'created_at' as created_at
FROM tweet
WHERE location_id LIKE 'germany%';
"""

all_tweets = db_to_pandas(sql_query)
all_tweets['date'] = pd.to_datetime(all_tweets['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
all_tweets['date'] = all_tweets.date.dt.normalize()

total_tweets = all_tweets['date'].value_counts()
total_tweets = total_tweets.to_frame().reset_index().rename(columns={'index': 'date', 'date': 'count'})
total_tweets.info
#total_tweets = total_tweets.reset_index()


total_tweets

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rcParams

sns.set(style="dark")
plt.title("DBPedia concepts identified in tweets in Germany")

#sns.set_context("talk")

# set size of figure
rcParams['figure.figsize'] = 15,7

# first axis
plot_df = interesting_df
ax1 = sns.lineplot(data=plot_df, x='date', y='count', hue='concepts', linewidth=2)
ax1.set(xlabel='Tweet created at', ylabel='Concept count')

# second axis
ax2 = plt.twinx()
sns.lineplot(data=total_tweets, x='date', y='count', color="black", ax=ax2, linewidth=2)
ax2.grid(False)
ax2.set(ylabel='Total tweets')
ax2.lines[0].set_linestyle("--")

plt.legend(labels=['Total tweets'])

plt.gcf().autofmt_xdate()